In [12]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

In [ ]:
class PrivateState(TypedDict):
    a: int
    b: int
    c: int 

class InputState(TypedDict):
    hello: str

class OutputState(TypedDict):
    bye: str

graph_builder = StateGraph(
    PrivateState, 
    input_schema=InputState, 
    output_schema=OutputState
    )

def node_one(state: InputState):
    print("node_one ->", state)
    return {
        "hello": "world"
    }

def node_two(state: PrivateState):
    print("node_two ->", state)
    return {
        "a": 1,
    }

def node_three(state: PrivateState):
    print("node_three ->", state)
    return {
        "b": 1,
    }

def node_four(state: PrivateState):
    print("node_four ->", state)
    return {
        "buy": "world",
    }


graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)





In [13]:
class State(TypedDict):
    hello: str
    
graph_builder = StateGraph(State) 

def node_one(state: State):
    print("node_one")

def node_two(state: State):
    print("node_two")

def node_three(state: State):
    print("node_three")

In [ ]:

graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", END)


In [ ]:
graph = graph_builder.compile()
graph.invoke(
    {"hello": "world"},
    )

node_one
node_two
node_three
